In [2]:
import open3d 
from numba.core.errors import NumbaDeprecationWarning,NumbaPendingDeprecationWarning, NumbaWarning
import warnings
warnings.simplefilter('ignore', category=NumbaDeprecationWarning)
warnings.simplefilter('ignore', category=NumbaWarning)
import sys
sys.path.append('/kaggle/code/ConeDetectionPointpillars')

from second.data.CustomNuscDataset import * #to register dataset
from models import * #to register model
import torch
from second.utils.log_tool import SimpleModelLog
from second.builder import target_assigner_builder, voxel_builder
from second.protos import pipeline_pb2
from second.pytorch.builder import (box_coder_builder, input_reader_builder,
                                    lr_scheduler_builder, optimizer_builder,
                                    second_builder)
from second.pytorch.core import box_torch_ops
import torchplus
import re
import time
from pathlib import Path
import numpy as np
import pandas as pd
import json
from collections import defaultdict
from google.protobuf import text_format
# from lyft_dataset_sdk.utils.geometry_utils import *
from lyft_dataset_sdk.lyftdataset import Quaternion
# from lyft_dataset_sdk.utils.data_classes import Box
from nuscenes.utils.geometry_utils import *
from nuscenes.utils.data_classes import Box

#commented second.core.non_max_suppression, nms_cpu, __init__.py, 
# pytorch.core.box_torch_ops line 524

In [4]:
from second.core import box_np_ops


TypeError: rotation_points_single_angle() missing 2 required positional arguments: 'points' and 'angle'

In [2]:
device = torch.device('cuda')

In [3]:
# import torch
# import torch.nn as nn
# dev = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
# t1 = torch.randn(1,2)
# t2 = torch.randn(1,2).to(dev)
# print(t1)  # tensor([[-0.2678,  1.9252]])
# print(t2)  # tensor([[ 0.5117, -3.6247]], device='cuda:0')
# t1.to(dev) 
# print(t1)  # tensor([[-0.2678,  1.9252]]) 
# print(t1.is_cuda) # False
# t1 = t1.to(dev)
# print(t1)  # tensor([[-0.2678,  1.9252]], device='cuda:0') 
# print(t1.is_cuda) # True

# class M(nn.Module):
#     def __init__(self):        
#         super().__init__()        
#         self.l1 = nn.Linear(1,2)

#     def forward(self, x):                      
#         x = self.l1(x)
#         return x
# model = M()   # not on cuda
# model.to(dev) # is on cuda (all parameters)
# print(next(model.parameters()).is_cuda) # True

In [4]:
config_path = './configs/cones_pp_initial.config'
from google.protobuf import text_format
import sys
sys.path.append('/kaggle/code/ConeDetectionPointpillars')
from second.protos import pipeline_pb2

config = pipeline_pb2.TrainEvalPipelineConfig()
with open(config_path, "r") as f:
    proto_str = text_format.Merge(f.read(), config)

In [5]:
print(proto_str)

model {
  second {
    network_class_name: "VoxelNet"
    voxel_generator {
      voxel_size: 0.10000000149011612
      voxel_size: 0.10000000149011612
      voxel_size: 4.0
      point_cloud_range: -120.0
      point_cloud_range: -60.0
      point_cloud_range: -4.0
      point_cloud_range: 80.0
      point_cloud_range: 60.0
      point_cloud_range: 5.0
      max_number_of_points_per_voxel: 15
    }
    voxel_feature_extractor {
      module_class_name: "PillarFeatureNet"
      num_filters: 64
      num_input_features: 5
    }
    middle_feature_extractor {
      module_class_name: "PointPillarsScatter"
      num_input_features: 64
      downsample_factor: 1
    }
    rpn {
      module_class_name: "RPNV2"
      layer_nums: 3
      layer_nums: 5
      layer_nums: 5
      layer_strides: 2
      layer_strides: 2
      layer_strides: 2
      num_filters: 64
      num_filters: 128
      num_filters: 256
      upsample_strides: 1.0
      upsample_strides: 2.0
      upsample_strides: 4.0
   

In [6]:
config_path = './configs/cones_pp_initial.config'
model_dir = f'./outputs/{time.time()}'
create_folder = False
measure_time = False
resume = False

In [7]:
## def run_train
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model_dir = str(Path(model_dir).resolve())
if create_folder:
    if Path(model_dir).exists():
        model_dir = torchplus.train.create_folder(model_dir)
model_dir = Path(model_dir)


# !sudo rm -R /kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs


if not resume and model_dir.exists():
    raise ValueError("model dir exists and you don't specify resume")

model_dir.mkdir(parents=True, exist_ok= True)
config_file_bkp = "pipeline.config"

if isinstance(config_path, str):
    config = pipeline_pb2.TrainEvalPipelineConfig()
    with open(config_path, "r") as f:
        proto_str = f.read()
        text_format.Merge(proto_str, config)
else:
    config = config_path
    proto_str = text_format.MessageToString(config, indent=2)

with (model_dir/config_file_bkp).open('w') as f:
    f.write(proto_str)
# Read config file
input_cfg = config.train_input_reader
eval_input_cfg = config.eval_input_reader
model_cfg = config.model.second # model's config
train_cfg = config.train_config # training config


In [8]:
def build_network(model_cfg, measure_time=False):
    voxel_generator = voxel_builder.build(model_cfg.voxel_generator)
    bv_range = voxel_generator.point_cloud_range[[0, 1, 3, 4]]
    box_coder = box_coder_builder.build(model_cfg.box_coder)
    target_assigner_cfg = model_cfg.target_assigner
    target_assigner = target_assigner_builder.build(target_assigner_cfg,
                                                    bv_range, box_coder)
    box_coder.custom_ndim = target_assigner._anchor_generators[0].custom_ndim
    net = second_builder.build(
        model_cfg, voxel_generator, target_assigner, measure_time=measure_time)
    return net

In [9]:
model_dir.exists()

True

In [10]:
net = build_network(model_cfg, measure_time).to(device)

In [11]:
net

VoxelNet(
  (voxel_feature_extractor): PillarFeatureNet(
    (pfn_layers): ModuleList(
      (0): PFNLayer(
        (linear): DefaultArgLayer(in_features=10, out_features=64, bias=False)
        (norm): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
      )
    )
  )
  (middle_feature_extractor): PointPillarsScatter()
  (rpn): RPNV2(
    (blocks): ModuleList(
      (0): Sequential(
        (0): ZeroPad2d(padding=(1, 1, 1, 1), value=0.0)
        (1): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(2, 2), bias=False)
        (2): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (3): ReLU()
        (4): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (5): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
        (6): ReLU()
        (7): DefaultArgLayer(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
 

In [12]:
target_assigner = net.target_assigner
voxel_generator = net.voxel_generator

In [13]:
print("num parameter: ", len(list(net.parameters())))
torchplus.train.try_restore_latest_checkpoints(model_dir, [net])

num parameter:  64


In [14]:
net_parallel = net

In [19]:
optimizer_cfg = train_cfg.optimizer
loss_scale = train_cfg.loss_scale_factor
fastai_optimizer = optimizer_builder.build(
    optimizer_cfg,
    net,
    mixed=False,
    loss_scale=loss_scale
)

False _amp_stash


In [23]:
print(net.voxel_feature_extractor)

PillarFeatureNet(
  (pfn_layers): ModuleList(
    (0): PFNLayer(
      (linear): DefaultArgLayer(in_features=10, out_features=64, bias=False)
      (norm): DefaultArgLayer(64, eps=0.001, momentum=0.01, affine=True, track_running_stats=True)
    )
  )
)


In [16]:
if loss_scale < 0:
    loss_scale = "dynamic"
if train_cfg.enable_mixed_precision:
    max_num_voxels = input_cfg.preprocess.max_number_of_voxels * input_cfg.batch_size
    print("max_num_voxels: %d" % (max_num_voxels))
    from apex import amp
    net, amp_optimizer = amp.initialize(net, fastai_optimizer, opt_level="O1",
                                        keep_batchnorm_fp32=None, loss_scale=loss_scale)
    net.metrics_to_float()
else:
    amp_optimizer = fastai_optimizer
torchplus.train.try_restore_latest_checkpoints(model_dir, [fastai_optimizer])
lr_scheduler = lr_scheduler_builder.build(optimizer_cfg, amp_optimizer, train_cfg.steps)

max_num_voxels: 70000
Selected optimization level O1:  Insert automatic casts around Pytorch functions and Tensor methods.

Defaults for this optimization level are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic
Processing user overrides (additional kwargs that are not None)...
After processing overrides, optimization options are:
enabled                : True
opt_level              : O1
cast_model_type        : None
patch_torch_functions  : True
keep_batchnorm_fp32    : None
master_weights         : None
loss_scale             : dynamic


In [17]:
def merge_second_batch(batch_list):
    if isinstance(batch_list[0], list):
        batch_list_c = []
        for example in batch_list:
            batch_list_c += example
        batch_list = batch_list_c

    example_merged = defaultdict(list)
    for example in batch_list:
        for k, v in example.items():
            example_merged[k].append(v)
    ret = {}
    for key, elems in example_merged.items():
        if key in [
            'voxels', 'num_points', 'num_gt', 'voxel_labels', 'gt_names', 'gt_classes', 'gt_boxes'
        ]:
            ret[key] = np.concatenate(elems, axis=0)
        elif key == 'metadata':
            ret[key] = elems
        elif key == "calib":
            ret[key] = {}
            for elem in elems:
                for k1, v1 in elem.items():
                    if k1 not in ret[key]:
                        ret[key][k1] = [v1]
                    else:
                        ret[key][k1].append(v1)
            for k1, v1 in ret[key].items():
                ret[key][k1] = np.stack(v1, axis=0)
        elif key == 'coordinates':
            coors = []
            for i, coor in enumerate(elems):
                coor_pad = np.pad(
                    coor, ((0, 0), (1, 0)), mode='constant', constant_values=i)
                coors.append(coor_pad)
            ret[key] = np.concatenate(coors, axis=0)
        elif key == 'metrics':
            ret[key] = elems
        else:
            ret[key] = np.stack(elems, axis=0)
    return ret


In [18]:

if train_cfg.enable_mixed_precision:
    float_dtype = torch.float16
else:
    float_dtype = torch.float32
multi_gpu = False
if multi_gpu:
    num_gpu = torch.cuda.device_count()
    print(f"MULTI_GPU: use {num_gpu} gpus")
    collate_fn = merge_second_batch_multigpu
else:
    collate_fn = merge_second_batch
    num_gpu = 1

In [19]:
dataset = input_reader_builder.build(
    input_cfg,
    model_cfg,
    training=True,
    voxel_generator=voxel_generator,
    target_assigner=target_assigner,
    multi_gpu=multi_gpu
)

feature_map_size [1, 600, 1000]
Loading NuScenes tables for version v1.0-trainval...
23 category,
8 attribute,
4 visibility,
64386 instance,
12 sensor,
10200 calibrated_sensor,
2631083 ego_pose,
68 log,
850 scene,
34149 sample,
2631083 sample_data,
1166187 sample_annotation,
4 map,
Done loading in 22.726 seconds.
Reverse indexing ...
Done reverse indexing in 5.3 seconds.


In [20]:
dataset[0]

INFO - 2021-01-20 22:47:38,190 - transforms - finding looplift candidates


{'voxels': array([[[  3.14525949,  -3.96943501,  -0.33711653,  -0.19019608,
            0.        ],
         [  3.15678115,  -3.93691871,  -0.33244276,  -0.20980392,
            0.        ],
         [  3.17456788,  -3.91114932,  -0.33074373,  -0.1745098 ,
            0.        ],
         ...,
         [  0.        ,   0.        ,   0.        ,   0.        ,
            0.        ],
         [  0.        ,   0.        ,   0.        ,   0.        ,
            0.        ],
         [  0.        ,   0.        ,   0.        ,   0.        ,
            0.        ]],
 
        [[ -9.4763381 ,  -0.47624819,  -0.46163136,  -0.44901961,
            0.        ],
         [ -9.48667664,  -0.42482655,  -0.46366235,  -0.44509804,
            0.        ],
         [  0.        ,   0.        ,   0.        ,   0.        ,
            0.        ],
         ...,
         [  0.        ,   0.        ,   0.        ,   0.        ,
            0.        ],
         [  0.        ,   0.        ,   0.       

In [21]:
def _worker_init_fn(worker_id):
    time_seed = np.array(time.time(), dtype=np.int32)
    np.random.seed(time_seed + worker_id)

In [22]:
num_gpu

1

In [23]:
dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=input_cfg.batch_size * num_gpu,
    shuffle=True,
    num_workers=input_cfg.preprocess.num_workers * num_gpu,
    pin_memory=False,
    collate_fn=collate_fn,
    worker_init_fn=_worker_init_fn,
    drop_last=not multi_gpu
)

In [24]:
dataloader.num_workers

3

In [25]:
model_logging = SimpleModelLog(model_dir)
model_logging.open()
model_logging.log_text(proto_str + "\n", 0, tag='config')
start_step = net.get_global_step()
total_step = train_cfg.steps
t = time.time()
steps_per_eval = train_cfg.steps_per_eval
clear_metrics_every_epoch = train_cfg.clear_metrics_every_epoch

amp_optimizer.zero_grad()
step_times = []
step = start_step

ave_valid_loss = 0.0

model: {
  second: {
    network_class_name: "VoxelNet"
    voxel_generator {
      full_empty_part_with_mean: false
      point_cloud_range : [-120, -60, -4 , 80, 60, 5] # will use [0,1,3,4] means [-100,-100,100,1000]
      voxel_size : [0.1, 0.1, 4]
      max_number_of_points_per_voxel : 15
      #block_filtering: true # filter voxels by block height
      #block_factor: 1 # height calc width: voxel_size * block_factor * block_size= (0.05 * 1 * 8)
      #block_size: 8
      #height_threshold: 0.2 # locations with height < height_threshold will be removed.
    }
    voxel_feature_extractor: {
      module_class_name: "PillarFeatureNet"
      num_filters: [64]
      with_distance: false
      num_input_features: 5
    }
    middle_feature_extractor: {
      module_class_name: "PointPillarsScatter"
      downsample_factor: 1
      num_input_features: 64
    }
    rpn: {
      module_class_name: "RPNV2"
      layer_nums: [3, 5, 5]
      layer_strides: [2, 2, 2]
      num_filters: [64, 12

In [26]:
def example_convert_to_torch(example, dtype=torch.float32,
                             device=None) -> dict:
    device = device or torch.device("cuda")
    example_torch = {}
    float_names = [
        "voxels", "anchors", "reg_targets", "reg_weights", "bev_map", "importance"
    ]
    for k, v in example.items():
        if k in ['gt_names', 'gt_classes', 'gt_boxes','points']:
            example_torch[k] = example[k]
            continue

        if k in float_names:
            # slow when directly provide fp32 data with dtype=torch.half
            example_torch[k] = torch.tensor(
                v, dtype=torch.float32, device=device).to(dtype)
        elif k in ["coordinates", "labels", "num_points"]:
            example_torch[k] = torch.tensor(
                v, dtype=torch.int32, device=device)
        elif k in ["anchors_mask"]:
            example_torch[k] = torch.tensor(
                v, dtype=torch.uint8, device=device)
        elif k == "calib":
            calib = {}
            for k1, v1 in v.items():
                calib[k1] = torch.tensor(
                    v1, dtype=dtype, device=device).to(dtype)
            example_torch[k] = calib
        elif k == "num_voxels":
            example_torch[k] = torch.tensor(v)
        else:
            example_torch[k] = v
    return example_torch


In [27]:
def time_to_str(t, mode='min'):
    if mode == 'min':
        t = int(t) / 60
        hr = t // 60
        min = t % 60
        return '%2d hr %02d m' % (hr, min)

    elif mode == 'sec':
        t = int(t)
        min = t // 60
        sec = t % 60
        return '%2d min %02d sec' % (min, sec)


    else:
        raise NotImplementedError


In [28]:
import tqdm
from tqdm import tqdm_notebook
display_step=50
clear_metrics_every_epoch = True
try:
    start_tic = time.time()
    print("num samples: %d" % (len(dataset)))
    while True:
        if clear_metrics_every_epoch:
            net.clear_metrics()
        for example in tqdm_notebook(dataloader):
            #             print(example)
            lr_scheduler.step(net.get_global_step())
            example.pop("metrics")
            example_torch = example_convert_to_torch(example, float_dtype)


            ret_dict = net_parallel(example_torch)
            loss = ret_dict["loss"].mean()
            cls_loss_reduced = ret_dict["cls_loss_reduced"].mean()
            loc_loss_reduced = ret_dict["loc_loss_reduced"].mean()


            if train_cfg.enable_mixed_precision:
                if net.get_global_step() < 100:
                    loss *= 1e-3
                with amp.scale_loss(loss, amp_optimizer) as scaled_loss:
                    scaled_loss.backward()
            else:
                loss.backward()
            torch.nn.utils.clip_grad_norm_(net.parameters(), 10.0)
            amp_optimizer.step()
            amp_optimizer.zero_grad()
            net.update_global_step()

            cls_preds = ret_dict["cls_preds"]
            labels = example_torch["labels"]
            cared = ret_dict["cared"]

            net_metrics = net.update_metrics(cls_loss_reduced,
                                             loc_loss_reduced, cls_preds,
                                             labels, cared)
            step_time = (time.time() - t)
            step_times.append(step_time)
            t = time.time()
            metrics = {}
            global_step = net.get_global_step()

            if global_step % display_step == 0:
                eta = time.time() - start_tic
                if measure_time:
                    for name, val in net.get_avg_time_dict().items():
                        print(f"avg {name} time = {val * 1000:.3f} ms")

                metrics["step"] = global_step
                metrics['epoch'] = global_step / len(dataloader)
                metrics['steptime'] = np.mean(step_times)
                # metrics["runtime"].update(time_metrics[0])
                metrics['valid'] = ave_valid_loss
                step_times = []

                metrics["loss"] = net_metrics['loss']['cls_loss'] + net_metrics['loss']['loc_loss']
                metrics["cls_loss"] = net_metrics['loss']['cls_loss']
                metrics["loc_loss"] = net_metrics['loss']['loc_loss']

                if model_cfg.use_direction_classifier:
                    dir_loss_reduced = ret_dict["dir_loss_reduced"].mean()
                    metrics["dir_rt"] = float(
                        dir_loss_reduced.detach().cpu().numpy())

                metrics['lr'] = float(amp_optimizer.lr)
                metrics['eta'] = time_to_str(eta)
                model_logging.log_metrics(metrics, global_step)

                net.clear_metrics()
            if global_step % steps_per_eval == 0:
                torchplus.train.save_models(model_dir, [net, amp_optimizer],
                                            net.get_global_step())
            step += 1
            if step >= total_step:
                break
        if step >= total_step:
            break
except Exception as e:
    example['metadata'] = 'None'
    model_logging.log_text(str(e), step)
    model_logging.log_text(json.dumps(example['metadata'], indent=2), step)
    torchplus.train.save_models(model_dir, [net, amp_optimizer], step)
    raise e
finally:
    model_logging.close()


num samples: 14856


<ipython-input-28-1993cac64d66>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for example in tqdm_notebook(dataloader):


  0%|          | 0/14856 [00:00<?, ?it/s]

INFO - 2021-01-20 22:47:41,429 - transforms - finding looplift candidates
INFO - 2021-01-20 22:47:43,251 - transforms - finding looplift candidates


Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 32768.0


INFO - 2021-01-20 22:47:55,250 - transforms - finding looplift candidates


step=50, epoch=0.003366, steptime=2.356, valid=0.0, loss=430.0, cls_loss=402.1, loc_loss=27.96, lr=0.0001001, eta= 0 hr 01 m
step=100, epoch=0.006731, steptime=2.289, valid=0.0, loss=110.4, cls_loss=84.14, loc_loss=26.28, lr=0.0001006, eta= 0 hr 03 m
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 16384.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 8192.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 4096.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 2048.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 1024.0
Gradient overflow.  Skipping step, loss scaler 0 reducing loss scale to 512.0
step=150, epoch=0.0101, steptime=2.355, valid=0.0, loss=52.78, cls_loss=29.65, loc_loss=23.14, lr=0.0001014, eta= 0 hr 05 m
step=200, epoch=0.01346, steptime=2.303, valid=0.0, loss=24.7, cls_loss=6.65, loc_loss=18.05, lr=0.0001024, eta= 0 hr 07 m
step=250, epoch=0.01683, ste

KeyboardInterrupt: 

In [29]:
torchplus.train.save_models(model_dir, [net, amp_optimizer],
                                            net.get_global_step())

In [32]:
model_dir

PosixPath('/kaggle/code/ConeDetectionPointpillars/customNuscenes/outputs/1611143208.1997967')

In [ ]:
dset = CustomNuscDataset()

In [ ]:
len(dset)

In [ ]:
dset[0]['lidar']['annotations']['boxes']

In [ ]:
min_x, max_x,min_y, max_y,min_z, max_z= 999999,-999999,999999,-999999,999999,-999999
from tqdm import tqdm_notebook
for i in tqdm_notebook(range(len(dset))):
    for box in dset[i]['lidar']['annotations']['boxes']:
        min_x = round(min(min_x, box[0]),2)
        max_x = round(max(max_x, box[0]),2)
        min_y = round(min(min_y, box[1]),2)
        max_y = round(max(max_y, box[1]),2)
        min_z = round(min(min_z, box[2]),2)
        max_z = round(max(max_z, box[2]),2)
    print(min_x, max_x,min_y, max_y,min_z, max_z)

In [ ]:
print(min_x, max_x,min_y, max_y,min_z, max_z)